In [2]:
import numpy as np
import cv2
import face_recognition
import os
from datetime import datetime
import imutils
import pytesseract
import random
pytesseract.pytesseract.tesseract_cmd = 'C:\Program Files\Tesseract-OCR\\tesseract'

path="train"
images=[]
ClassNames=[]

mylist=os.listdir(path)
for cls in mylist:
    currentImg=cv2.imread(f"{path}/{cls}")
    images.append(currentImg)
    ClassNames.append(os.path.splitext(cls)[0])
# print(ClassNames)
# cv2.imshow("image",images[2])
# cv2.waitKey(0)


def get_number_plate(image,name):
#     image = cv2.imread(images)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image = cv2.bilateralFilter(gray_image, 11, 17, 17) 
    edged = cv2.Canny(gray_image, 30, 200) 
    cnts,new = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    image1=image.copy()
    cv2.drawContours(image1,cnts,-1,(0,255,0),3)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True) [:30]
    screenCnt = None
    image2 = image.copy()
    cv2.drawContours(image2,cnts,-1,(0,255,0),3)

    for c in cnts:
        perimeter = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.018 * perimeter, True)
        if len(approx) == 4: 
                screenCnt = approx
        x,y,w,h = cv2.boundingRect(c) 
        new_img=image[y:y+h,x:x+w]
        i=random.randint(0,1)
        cv2.imwrite('./'+str(name)+'.png',new_img)
        break
    cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 3)
    Cropped_loc = f'./{name}.png'
    plate = pytesseract.image_to_string(Cropped_loc, lang='eng')
    number_plate=plate[:-1]
    return number_plate





def encoder(images):
    encode_list=[]
    for img in images:
        img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode= face_recognition.face_encodings(img)[0]
        encode_list.append(encode)
    return encode_list

def mark_attendence(name,num):
    with open("entry.csv", 'r+') as f:
        myDataList=f.readlines()
        nameList=[]
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now= datetime.now()
            datestring=now.strftime('%H:%M:%S')
            f.writelines(f"\n{name},{num},{datestring}")
            
#         print(myDataList)

    


encodeList= encoder(images)
# print(len(encodeList))
print("Encoding Complete")



cap=cv2.VideoCapture(0)

while True:
    success, img = cap.read()

# img= cv2.imread("data/shahrukh.jpg")

    imgS=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    face_Current_Frame= face_recognition.face_locations(imgS)
    encoding_Current_Frame= face_recognition.face_encodings(imgS,face_Current_Frame)

    for encode_face, face_loc in zip(encoding_Current_Frame,face_Current_Frame):
        matches=face_recognition.compare_faces(encodeList,encode_face)
        face_Distance=face_recognition.face_distance(encodeList,encode_face)
        match_index=np.argmin(face_Distance)
        if matches[match_index]:
            name=ClassNames[match_index].upper() 
            plate=get_number_plate(img,name)
            mark_attendence(name,plate)
    #             print(f"Attandence Marked for {name}")
            y1,x2,y2,x1=face_loc
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),1)
            cv2.rectangle(img,(x1,y2-10),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img, name,(x1+2,y2-2),cv2.FONT_HERSHEY_COMPLEX,0.3,(255,255,255),1)


    cv2.imshow("video",img)
# cv2.waitKey(0)
    if cv2.waitKey(1) & 0xff ==ord('q'):
        break


Encoding Complete


-1

In [4]:
cv2.imwrite("image2.jpg",img)

True